In [ ]:
rules = [
    {"item": "brown", "sem": {"color": "BROWN"}},
    {"item": "dog", "sem": {"entity": "DOG"}},

    {"when":    {"input": "test"},
     "then":    [{"#lookup": "$brown.sem", "into": "?sem-art"},
                {"#lookup": "$dog.sem", "into": "?sem-noun"},
                {"cat": "np", "sem": {"#combine": ["?sem-noun", "?sem-art"]} }]
    },

    {"sem-2": "#\\ ?semh + ?semc"}
]